In [1]:
import pandas as pd
import numpy as np
import re
import string
import nltk.data
from transformers import BertTokenizer, BertModel, BertForMaskedLM, AdamW, get_linear_schedule_with_warmup
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

In [2]:
if torch.cuda.is_available():
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU: ', torch.cuda.get_device_name(0))
    device = torch.device("cuda:0")
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU:  GeForce RTX 2070 with Max-Q Design


In [3]:
data = pd.read_csv('./wiki_movie_plots_deduped.csv')

In [4]:
data.head()

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot
0,1901,Kansas Saloon Smashers,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Kansas_Saloon_Sm...,"A bartender is working at a saloon, serving dr..."
1,1901,Love by the Light of the Moon,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Love_by_the_Ligh...,"The moon, painted with a smiling face hangs ov..."
2,1901,The Martyred Presidents,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/The_Martyred_Pre...,"The film, just over a minute long, is composed..."
3,1901,"Terrible Teddy, the Grizzly King",American,Unknown,NaN,unknown,"https://en.wikipedia.org/wiki/Terrible_Teddy,_...",Lasting just 61 seconds and consisting of two ...
4,1902,Jack and the Beanstalk,American,"George S. Fleming, Edwin S. Porter",NaN,unknown,https://en.wikipedia.org/wiki/Jack_and_the_Bea...,The earliest known adaptation of the classic f...


In [5]:
plots = data['Plot'].tolist()

In [6]:
print(len(plots))

34886


In [7]:
model_version = 'bert-base-uncased'
wordTokenizer = BertTokenizer.from_pretrained(model_version, do_lower_case=model_version.endswith("uncased"))
sentTokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

In [8]:
printable = set(string.printable)
MAX_LEN = 128

def clean(plot):
  plot1 = ''.join(filter(lambda x: x in printable, plot))
  plot1 = re.sub('\[\d*\]', '', plot1)
  return plot1

def addMask(ids, wordTokenizer):
  # l = original length before padding
  label = []
  maskid = wordTokenizer.convert_tokens_to_ids('[MASK]')
  for i, id in enumerate(ids):
    if wordTokenizer.convert_ids_to_tokens(id) == '[CLS]':
      label.append(-1)
      continue
    if wordTokenizer.convert_ids_to_tokens(id) == '[SEP]':
      break
    if 0.15 >= np.random.random():
      p = np.random.random()
      if 0.8 >= p:
        label.append(id)
        ids[i] = maskid
      elif 0.9 >= p:
        newid = np.random.randint(0, len(wordTokenizer.vocab)-1)
        ids[i] = newid
        label.append(-1)
      else: 
        label.append(-1)
    else:
      label.append(-1)
  while len(label) < len(ids):
    label.append(-1)
  assert len(label) == 128
  return ids, label
  

def preprocess(plots):
  ids = []
  attentionMasks = []
  labels = []
  for plot in plots:
    plot = clean(plot)
    sents = sentTokenizer.tokenize(plot)

    for sent in sents:
      # print(sent)
      encodedDict = wordTokenizer.encode_plus(sent,add_special_tokens=True,max_length=MAX_LEN,pad_to_max_length=True)
      labels.append(encodedDict['input_ids'].copy())
      inputid, _ = addMask(encodedDict['input_ids'], wordTokenizer)
      ids.append(inputid)
      attentionMasks.append(encodedDict['attention_mask'])
      # labels.append(label)
      # print(inputid)
      # print(encodedDict['attention_mask'])
      # print(label)
      # print(wordTokenizer.convert_ids_to_tokens(encodedDict['input_ids']))
  
  return ids, attentionMasks, labels

In [9]:
ids, masks, labels = preprocess(plots[:25000])

In [10]:
from sklearn.model_selection import train_test_split

train_inputs,validation_inputs,train_masks,validation_masks = train_test_split(ids, masks, random_state=42, test_size=0.1)
train_labels,validation_labels,_,_ = train_test_split(labels,ids,random_state=42,test_size=0.1)

In [11]:
train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)
train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

In [12]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

batch_size = 16

train_data = TensorDataset(train_inputs,train_masks,train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data,sampler=train_sampler,batch_size=batch_size)

validation_data = TensorDataset(validation_inputs,validation_masks,validation_labels)
validation_sampler = RandomSampler(validation_data)
validation_dataloader = DataLoader(validation_data,sampler=validation_sampler,batch_size=batch_size)

In [13]:
model_version = 'bert-base-uncased'
model = BertForMaskedLM.from_pretrained(model_version)
model.to(device)

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [14]:
lr = 2e-5
adam_epsilon = 1e-8

# Number of training epochs (authors recommend between 2 and 4)
epochs = 3

num_warmup_steps = 0
num_training_steps = len(train_dataloader)*epochs

### In Transformers, optimizer and schedules are splitted and instantiated like this:
optimizer = AdamW(model.parameters(), lr=lr,eps=adam_epsilon,correct_bias=False)  # To reproduce BertAdam specific behavior set correct_bias=False
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=num_training_steps)  # PyTorch scheduler

In [15]:
train_loss_set = []
learning_rate = []

# Gradients gets accumulated by default
model.zero_grad()

# tnrange is a tqdm wrapper around the normal python range
for epoch_i in range(1,epochs+1):
  print('======== Epoch {:} / {:} ========'.format(epoch_i, epochs))
  # Calculate total loss for this epoch
  train_loss = 0
  # Set our model to training mode (as opposed to evaluation mode)
  model.train()
  for step, batch in enumerate(train_dataloader):
    # Add batch to GPU
    b_input_ids = batch[0].to(device)
    b_input_mask = batch[1].to(device)
    b_labels = batch[2].to(device)
    
    model.zero_grad()

    # Forward pass
    loss, logits = model(b_input_ids, attention_mask=b_input_mask, masked_lm_labels=b_labels)
    
    # Backward pass
    loss.backward()
    
    # Clip the norm of the gradients to 1.0
    # Gradient clipping is not in AdamW anymore
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
    
    # Update parameters and take a step using the computed gradient
    optimizer.step()
    
    # Update learning rate schedule
    scheduler.step()

    # Clear the previous accumulated gradients
    optimizer.zero_grad()
    
    # Update tracking variables
    train_loss += loss.item()

  # Calculate the average loss over the training data.
  avg_train_loss = train_loss / len(train_dataloader)

  #store the current learning rate
  for param_group in optimizer.param_groups:
    print("\n\tCurrent Learning rate: ",param_group['lr'])
    learning_rate.append(param_group['lr'])
    
  train_loss_set.append(avg_train_loss)
  print(F'\n\tAverage Training loss: {avg_train_loss}')
    
  # Validation

  # Put model in evaluation mode to evaluate loss on the validation set
  model.eval()

  # Tracking variables 
  validation_loss = 0.0

  # Evaluate data for one epoch
  for batch in validation_dataloader:
    # Add batch to GPU
    # batch = tuple(t.to(device) for t in batch)
    # # Unpack the inputs from our dataloader
    # b_input_ids, b_input_mask, b_labels = batch
    b_input_ids = batch[0].to(device)
    b_input_mask = batch[1].to(device)
    b_labels = batch[2].to(device)
    # Telling the model not to compute or store gradients, saving memory and speeding up validation
    with torch.no_grad():
      # Forward pass, calculate logit predictions
      loss, logits = model(b_input_ids, attention_mask=b_input_mask, masked_lm_labels=b_labels)
    
    # Move logits and labels to CPU
    validation_loss += loss.item()
  avg_validation_loss = validation_loss / len(validation_dataloader)

  print(F'\n\tAverage Validation loss: {avg_validation_loss}')

  torch.save(model, './model.pt')

======== Epoch 1 / 3 ========
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269


KeyboardInterrupt: 